In [ ]:
import pandas as pd
import numpy as np
import json
import nltk
import re
import csv
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [ ]:
%matplotlib inline
pd.set_option('display.max_colwidth', 300)

In [ ]:
meta = pd.read_excel("articoli_testo_new_2.xlsx", encoding="utf8", error_bad_lines=False)
meta = meta.replace(np.nan, '', regex=True)

In [ ]:
article_id = meta['ID']
len(article_id)

In [ ]:
meta['Trend 1'][0]

In [ ]:
tech1 = []
for i in meta['Trend 1']:
    tech1.append(i)

tech2 = []
for i in meta['Trend 2']:
    tech2.append(i)
    
tech3 = []
for i in meta['Trend 3']:
    tech3.append(i)

In [ ]:
Tech = []
for i in range(len(article_id)):
    l = []
    if len(tech1[i]): l.append(tech1[i])
    #if len(tech2[i]): l.append(tech2[i])
    #if len(tech3[i]): l.append(tech3[i])
        
    #Tech.append(list(l))
    Tech.append(', '.join(map(str, list(l)))) 
    
Tech[0]

In [ ]:
len(article_id)

In [ ]:
# Create Tech Dataframe
tech = pd.DataFrame({'ID': article_id, 'Tech': Tech})
tech.head()

In [ ]:
tech.describe()

In [ ]:
articoli = meta[['ID','Titolo','Abstract', 'Testo']]
articoli = pd.merge(articoli, tech, on = 'ID')

In [ ]:
articoli.head()

In [ ]:
len(articoli)

In [ ]:
# remove samples with no tags
articoli['Tech'] = articoli['Tech'].apply(lambda y: '' if len(y)==0 else y)
articoli = articoli[~(articoli['Tech'].str.len() == 0)]
#articoli = articoli[~(articoli['Abstract'].str.len() == 0)]
articoli = articoli[~(articoli['Testo'].str.len() == 0)]

In [ ]:
len(articoli)

In [ ]:
articoli.head()

In [ ]:
# function for text cleaning 
def clean_text(text):
    # remove backslash-apostrophe 
    text = re.sub("\'", "", text)
    text = re.sub(r'[?|!|\'|"|#]',r'',text)
    text = re.sub(r'[.|,|)|(|\|/]',r' ',text)
    # remove everything except alphabets 
    text = re.sub("[^a-zA-Z]"," ",text) 
    
    text = text.replace("\n"," ")
    
    # remove whitespaces 
    text = ' '.join(text.split()) 
    # convert text to lowercase 
    text = text.lower() 
    
    return text

In [ ]:
articoli['Titolo'] = articoli['Titolo'].apply(lambda x: clean_text(x))
articoli['Abstract'] = articoli['Abstract'].apply(lambda x: clean_text(x))
articoli['Testo'] = articoli['Testo'].apply(lambda x: clean_text(x))
articoli.head()

In [ ]:
len(articoli['ID'])

In [ ]:
plt.figure(figsize=(12,15))
ax=sns.countplot(articoli["Tech"])
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
plt.show()

In [ ]:
len(articoli['Tech'].unique())

In [ ]:
def freq_words(x, terms = 30): 
  all_words = ' '.join([text for text in x]) 
  all_words = all_words.split() 
  fdist = nltk.FreqDist(all_words) 
  words_df = pd.DataFrame({'word':list(fdist.keys()), 'count':list(fdist.values())}) 
  
  # selecting top 20 most frequent words 
  d = words_df.nlargest(columns="count", n = terms) 
  
  # visualize words and frequencies
  plt.figure(figsize=(12,15)) 
  ax = sns.barplot(data=d, x= "count", y = "word") 
  ax.set(ylabel = 'Word') 
  plt.show()
  
# print 100 most frequent words 
freq_words(articoli['Testo'], 30)

In [ ]:
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# function to remove stopwords
def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

articoli['Abstract'] = articoli['Abstract'].apply(lambda x: remove_stopwords(x))
articoli['Titolo'] = articoli['Titolo'].apply(lambda x: remove_stopwords(x))
articoli['Testo'] = articoli['Testo'].apply(lambda x: remove_stopwords(x))

In [ ]:
#articoli['Txt'] = articoli['Titolo'] + articoli['Abstract']
articoli['Txt'] = articoli['Titolo'] + articoli['Testo'] + articoli['Abstract']

In [ ]:
del(articoli['Abstract'])
del(articoli['Titolo'])
del(articoli['Testo'])

In [ ]:
articoli.head()

In [ ]:
freq_words(articoli['Txt'], 30)

In [ ]:
tags = articoli['Tech'].apply(lambda x:len(x.split(',')))

In [ ]:
sns.countplot(x=tags)

In [ ]:
import nltk
from nltk.tokenize import ToktokTokenizer
x=nltk.FreqDist(ToktokTokenizer().tokenize(articoli['Txt']))
plt.figure(figsize=(16,5))
x.plot(20)

In [ ]:
# https://stackabuse.com/text-classification-with-python-and-scikit-learn/
# INTERESTING FOR SAVING MODEL
from pandas import Series
y = Series(articoli['Tech']).values

# convert text to features
from sklearn.preprocessing import MultiLabelBinarizer

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(articoli['Tech'])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = tfidfconverter.fit_transform(articoli['Txt']).toarray()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train, y_train) 

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_pred))

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
print(accuracy_score(y_test, y_pred))

In [ ]:
# predict
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

In [ ]:
#build prediction model
from sklearn.linear_model import LogisticRegression

# Binary Relevance
from sklearn.multiclass import OneVsRestClassifier

# Performance metric
from sklearn.metrics import f1_score

In [ ]:
lr = LogisticRegression()
clf = OneVsRestClassifier(lr)

X_train, X_test, y_train, y_test

# fit model on train data
clf.fit(X_train, y_train)
    
# make predictions for validation set
y_pred = clf.predict(X_test)

In [ ]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

In [ ]:
y_test[3]

In [ ]:
def infer_tags(q):
    print (q)
    #q = clean_text(q)
    #q = remove_stopwords(q)
    q_vec = tfidfconverter.transform([q])
    q_pred = clf.predict(q_vec)
    return multilabel_binarizer.inverse_transform(q_pred)

In [ ]:
for i in range(5): 
    k = X_test.values(1).index[0]
    print("Article: ", articoli['Abstract'][k], \
        "\nPredicted tech: ", infer_tags(X_test[k])),
    print("Actual tech: ",articoli['Tech'][k], "\n") 

In [ ]:
infer_tags(X_test[2])

In [ ]:
k = 5
print(articoli['ID'][k])
print (articoli['Tech'][k])
print(infer_tags[X_test[k]])

In [ ]:
len(articoli['Tech'].unique())